In [192]:
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import regex as re
import requests
import pandas as pd

Fetch HTML of Main Website

In [193]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://mrcc.purdue.edu/research/awssi/indexAwssi.jsp")
driver.find_element("id", "showStations").click()
soup = BeautifulSoup(driver.page_source)
driver.quit()


Fetch Links of All End-of-Year Station Data

In [194]:
re.findall(r'rawssi([^"]*)',str(soup.find_all(id="stations")[0]))
csv_links = []
for i in re.findall(r'rawssi([^"]*)',str(soup.find_all(id="stations")[0])):
    z = "https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi" + str(i)
    csv_links.append(z)

csv_links


['https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.BHMthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.HSVthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.MOBthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.MGMthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.MSLthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.TCL.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.ANC.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.BTTthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.USC00501240.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.MPK.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.FAIthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.JNUthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.LPW.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.ORT.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.TAL.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.BRWthr.csv',
 'https://mrcc.purdue.edu/AWSSI/RAWSSI/rawssi.FLGthr.csv',
 'https://

Creating Main Dataframe

In [205]:
columns = ["Season", "Start", "End", "Length", "Msng", "Tmax", "Tmin", "TempSc", "Sfall", "Sdepth", "SnowSc", "AWSSI", "Events", "RAWSSI", "Location"]
data = pd.DataFrame(columns=columns)
for x in csv_links:
    df = pd.read_csv(x, skiprows=1)
    df2 = pd.read_csv(x)
    df['Location']=re.findall(r'"([^"]*)' ,df2.columns[2])[0]
    df.rename(columns={"   Season":"Season", "     Start":"Start", "  Msng": "Msng", "  Tmax": "Tmax", "  Tmin":"Tmin", " Sfall":"Sfall", "       End":"End", " AWSSI":"AWSSI"}, inplace=True)
    data= pd.concat([data,df])

In [206]:
data = data.loc[data["AWSSI"] >= 0]
data.to_csv("Winter-Data.csv")
#data.drop(columns="Location", inplace=True)
x= data["Location"].str.split(pat=" - ")
x = x.tolist()
y = pd.DataFrame(x, columns = ["State", "City"])
y
data=pd.concat([data.reset_index(drop=True),y.reset_index(drop=True)], axis =1).drop(columns="Location")

In [207]:
#Abbreviations to state courtesy of https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
}
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))


for code in data["State"].unique():
    data.loc[data["State"] == code, "State"] = abbrev_to_us_state[code]


In [210]:

for city in data["City"].unique():
    if(any(re.findall(r'\(', str(city)))):
        data.loc[data["City"] == city, "City"] = re.findall(r'([^ (]*)', str(city))[0]
    if(any(re.findall(r'-', city))):
       data.loc[data["City"] == city, "City"] = re.findall(r'([^-]*)', str(city))[0]
    if(any(re.findall(r'St ', city))):
        data.loc[data["City"] == city, "City"] = "St. " + re.findall(r'St (.*)', str(city))[0]

,Season,Start,End,Length,Msng,Tmax,Tmin,TempSc,Sfall,Sdepth,SnowSc,AWSSI,Events,RAWSSI,State,City
0,1950-1951,1950-11-24,1951-02-28,97,0,3,88,91,4,1,5,96,3,17,Alabama,Birmingham
1,1951-1952,1951-12-01,1952-02-29,91,0,0,38,38,0,0,0,38,0,0,Alabama,Birmingham
2,1952-1953,1952-12-01,1953-02-28,90,0,0,34,34,0,0,0,34,0,0,Alabama,Birmingham
3,1953-1954,1953-12-01,1954-02-28,90,0,1,52,53,0,0,0,53,0,0,Alabama,Birmingham
4,1954-1955,1954-12-01,1955-02-28,90,0,1,61,62,0,0,0,62,0,0,Alabama,Birmingham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22733,2015-2016,2015-11-04,2016-05-08,187,0,123,579,702,140,370,510,1212,42,323,Wyoming,Wamsutter
22734,2016-2017,2016-11-17,2017-05-18,183,0,115,511,626,86,289,375,1001,28,202,Wyoming,Wamsutter
22735,2017-2018,2017-09-19,2018-04-21,215,0,42,539,581,78,149,227,808,26,175,Wyoming,Wamsutter
22736,2018-2019,2018-10-10,2019-05-23,226,0,144,718,862,116,631,747,1609,39,251,Wyoming,Wamsutter


In [261]:
censusdata = pd.read_csv("sub-est2019_all.csv", encoding = "ISO-8859-1")
censusdata.loc[censusdata["NAME"].str.contains("Birmingham") & censusdata["STNAME"].str.contains("Alabama"), "CENSUS2010POP"].reset_index(drop=True)
data["2010Pop"] = 0


for state in data["State"].unique():
    for city in data.loc[data["State"] == state, "City"].unique():
        if (any(censusdata.loc[censusdata["NAME"].str.contains(city) & censusdata["STNAME"].str.contains(state), "CENSUS2010POP"].reset_index(drop=True))):
            data.loc[data["City"] == city, "2010Pop"] =  censusdata.loc[censusdata["NAME"].str.contains(city) & censusdata["STNAME"].str.contains(state), "CENSUS2010POP"].reset_index(drop=True)[0]
        else:
            data.loc[data["City"] == city, "2010Pop"] = 0
        #print(censusdata.loc[censusdata["NAME"].str.contains(city) & censusdata["STNAME"].str.contains(state), "CENSUS2010POP"].reset_index(drop=True)[0])

In [264]:
data.to_csv("Winter-And-Pop.csv")